In [1]:
import numpy as np
import random
import pickle
from numba import jit
import time

In [2]:
from game.game_of_life import gol, rle

In [3]:
game_size = 512
player_size = 64
experiment_name = "Random King of Hill" 
author = "Emil Wallner Githhub: ewallner" 
comments = "This experiment allows us to compare how smart an algorithm is compared to brute force"

In [4]:
#@jit
def get_players(player_size):
    dist_1 = random.random()
    dist_2 = random.random()
    
    player_1_value = [1, 0]
    player_2_value = [2, 0]
    
    player_1_program = np.random.choice(player_1_value, (player_size, player_size), p=[dist_1, (1.0 - dist_1)])
    player_2_program = np.random.choice(player_2_value, (player_size, player_size), p=[dist_2, (1.0 - dist_2)])
    
    return player_1_program, player_2_program

In [5]:
#@jit
def add_player_to_board(board, board_size, player_size, program, x, y):
    for b_x in range(player_size):
        for b_y in range(player_size):
            board[b_x + x][b_y + y] = program[b_x][b_y]
    return board

In [6]:
#@jit
def initiate_board(board_size, player_size):
    game_board = np.zeros((board_size, board_size), dtype=np.int8)
    
    p1_program, p2_program = get_players(player_size)
    
    grid_size_x = (game_size - (player_size * 2)) / 3
    grid_size_y = (game_size - player_size) / 2
    
    x_1 = int(grid_size_x)
    y_1 = int(grid_size_y)
    x_2 = int(x_1 + grid_size_x)
    y_2 = int(grid_size_y)
    
    game_board = add_player_to_board(game_board, board_size, player_size, p1_program, x_1, y_1)
    game_board = add_player_to_board(game_board, board_size, player_size, p2_program, x_2, y_2)
    
    return game_board, p1_program, p2_program

In [7]:
game_board, p1_program, p2_program = initiate_board(game_size, player_size)

In [8]:
for i in range(10):
    start_time = time.time()
    game_board, p1_program, p2_program = initiate_board(game_size, player_size)
    game = gol.GameOfLife(512, 512)
    game.set_grid(bytearray(game_board.tobytes()))
    game.run_steps(100)
    print("--- %s seconds ---" % (time.time() - start_time))

--- 0.23786187171936035 seconds ---
--- 0.24723005294799805 seconds ---
--- 0.23813891410827637 seconds ---
--- 0.23526310920715332 seconds ---
--- 0.2397308349609375 seconds ---
--- 0.23963594436645508 seconds ---
--- 0.23666095733642578 seconds ---
--- 0.2322838306427002 seconds ---
--- 0.23583507537841797 seconds ---
--- 0.24450302124023438 seconds ---


In [9]:
game_ints = [x for x in game.grid()]

In [10]:
@jit
def decide_winner(game_result):
    ones = 0
    twos = 0
    winner = 1
    
    for i in game_result:
        if i == 1:
            ones += 1
        elif i == 2:
            twos += 1
            
    if twos > ones:
        winner = 2
    return winner

print(decide_winner(game_ints))

1
